In [ ]:
import os
import re
import sys
import glob
import pickle
import tables
from collections import OrderedDict

import numpy as np
import matplotlib
from matplotlib import rcParams
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow import keras

if '..' not in sys.path:
    sys.path.append('..')
from dlml.utils import collect_experiments
from dlml.data import load_data_slide
from dlml.nn import predict

%matplotlib inline

### Load the network that will be used in the following to predict the values of momentum

In [ ]:
area_ID = 1
area_measure = 'momentum'
stoch_load_bus_IDs = [3]
rec_bus_IDs = [3, 14, 17]
additional_tags = ['ReLU_none', 'PowerFactory']

experiments = collect_experiments(area_ID, area_measure=area_measure, D=None, DZA=None, \
                                  stoch_load_bus_IDs=stoch_load_bus_IDs, H_G1=None, \
                                  rec_bus_IDs=rec_bus_IDs, additional_tags=additional_tags, \
                                  verbose=True)
experiment_IDs = list(experiments.keys())
experiment_ID = experiment_IDs[np.argmin([expt['val_loss'].min() for expt in experiments.values()])]
MAPE = experiments[experiment_ID]['MAPE']
loss = experiments[experiment_ID]['loss']
val_loss = experiments[experiment_ID]['val_loss']
batch_loss = experiments[experiment_ID]['batch_loss']
tags = experiments[experiment_ID]['tags']
print(f'The best experiment is {experiment_ID[:6]} (val_loss = {val_loss.min():.4f}, MAPE = {MAPE:.4f}%).')

experiments_path = '../experiments/neural_network/'
checkpoint_path = experiments_path + experiment_ID + '/checkpoints/'
checkpoint_files = glob.glob(checkpoint_path + '*.h5')
network_parameters = pickle.load(open(experiments_path + experiment_ID \
                                      + '/parameters.pkl', 'rb'))
epochs = [int(os.path.split(file)[-1].split('.')[1].split('-')[0]) for file in checkpoint_files]
best_idx = np.argmin(val_loss)
best_checkpoint = checkpoint_files[epochs.index(best_idx + 1)]
model = keras.models.load_model(best_checkpoint, compile=False)
model.compile()
data_dirs = ['..' + os.path.sep +
             os.path.sep.join([d for d in data_dir.split(os.path.sep) if '{}' not in d])
             for data_dir in network_parameters['data_dirs']]
# we need mean and standard deviation of the training set to normalize the data
x_train_mean = network_parameters['x_train_mean']
x_train_std  = network_parameters['x_train_std']
data_dir = data_dirs[0]
tmp = [re.findall('.*_bus', var_name)[0] for var_name in network_parameters['var_names']]
var_names_fmt = list(OrderedDict({k + '{}': [] for k in tmp}).keys())
if len(rec_bus_IDs) == 0:
    rec_bus_IDs = list(np.unique([int(re.findall('\d+', var_name)[0]) \
                                  for var_name in network_parameters['var_names']]))
    rec_bus_list = 'buses_' + '-'.join(map(str, rec_bus_IDs))
if not os.path.isdir(data_dir):
    raise Exception(f'{data_dir}: no such directory')
print(f'Loaded network from {best_checkpoint}.')
print(f'Data directory is {data_dir}.')
print(f'Variable names: {var_names_fmt}')

#### Get the list of files to use

In [ ]:
data_folder = '/home/daniele/Research/deep-power/data/IEEE39/PowerFactory'

split_generators = False

if split_generators:
    S_G03a, S_G03b = 400, 400 # [MVA]
    S_G03a, S_G03b = 500, 500 # [MVA]
    S_G03a, S_G03b = 800, 200 # [MVA]
    suffix = ''
    suffix = '_compensator'
    data_files = sorted(glob.glob(data_folder + f'/IEEE39_rnd_load_split_gen_*_{S_G03a}_{S_G03b}{suffix}.h5'))
else:
    S_G03 = 800 # [MVA]
    data_files = sorted(glob.glob(data_folder + f'/IEEE39_rnd_load_5.000*6_3.570*.h5'))

if len(data_files) == 0:
    raise Exception('No files match the pattern')

#### Load the data

In [ ]:
fids = [tables.open_file(f) for f in data_files]
time = fids[0].root.time.read()
pars = [fid.root.parameters.read() for fid in fids]
if split_generators:
    element_names = 'G02', 'G03a', 'G03b'
else:
    element_names = 'G02', 'G03'
Pe = [{elem: np.squeeze(fid.root[f'Pe_{elem}'].read()) for elem in element_names} for fid in fids]
Qe = [{elem: np.squeeze(fid.root[f'Qe_{elem}'].read()) for elem in element_names} for fid in fids]
bus_IDs = 3, 10, 14, 17, 39
Vd = [{ID: np.squeeze(fid.root[f'Vd_bus{ID}'].read()) for ID in bus_IDs} for fid in fids]
Vq = [{ID: np.squeeze(fid.root[f'Vq_bus{ID}'].read()) for ID in bus_IDs} for fid in fids]
line_IDs = ((10, 11), (10, 13), (14, 15), (16, 17), (1, 39), (3, 4))
try:
    Iline = [{f'{a}_{b}': np.squeeze(fid.root[f'I_line_{a}_{b}'].read()) for a,b in line_IDs} for fid in fids]
except:
    Iline = None
    print('No line currents saved in data files.')
for fid in fids:
    fid.close()

#### Compute the values of area 1 momentum

In [ ]:
def momentum(S, H, fn=60.):
    M = 0
    for k in S:
        M += S[k] * H[k]
    return 2 * M / fn * 1e-9

generator_IDs = list(map(lambda s: s.decode('utf-8'), np.squeeze(pars[0]['generator_IDs'])))
S = [{k: v*1e6 for k,v in zip(generator_IDs, np.squeeze(p['S_nominal']))} for p in pars]
H = [{k: v for k,v in zip(generator_IDs, np.squeeze(p['inertia']))} for p in pars]
if split_generators:
    area1_generator_IDs = ['G 02', 'G 03a', 'G 03b']
else:
    area1_generator_IDs = ['G 02', 'G 03']
S_area1 = {gen_ID: S[0][gen_ID] for gen_ID in area1_generator_IDs}
H_area1 = [{gen_ID: h[gen_ID] for gen_ID in area1_generator_IDs} for h in H]
M = np.array([momentum(S_area1, h) for h in H_area1])
print(M)

#### Plot the dynamics of the active and reactive powers of the split generators

In [ ]:
if split_generators:
    gen_IDs = 'G03a', 'G03b'
    h = 2
else:
    gen_IDs = 'G03',
    h = 3
n_gen = len(gen_IDs)
fig,ax = plt.subplots(n_gen, 2, sharex=True, figsize=(10, n_gen * h), squeeze=False)
cmap = 'kr'
Pstd, Qstd = 0, 0
for i,(P,Q) in enumerate(zip(Pe, Qe)):
    for j,gen_ID in enumerate(gen_IDs):
        if P[gen_ID].std() > Pstd:
            Pstd = P[gen_ID].std()
        if Q[gen_ID].std() > Qstd:
            Qstd = Q[gen_ID].std()
        ax[j,0].plot(time, P[gen_ID], lw=1, color=cmap[i], label=f'M = {M[i]:.2f}' + r' $GW\cdot s^2$')
        ax[j,1].plot(time, Q[gen_ID], lw=1, color=cmap[i])
        ax[j,0].set_ylabel(r'$P_{' + gen_ID + r'}$ [MW]')
        ax[j,1].set_ylabel(r'$Q_{' + gen_ID + r'}$ [MW]')
        for side in 'right','top':
            ax[j,i].spines[side].set_visible(False)
        ax[j,i].grid(which='major', axis='y', lw=0.5, ls=':', color=[.6,.6,.6])
xlim = [0, 60]
for a in ax[-1,:]:
    a.set_xlabel('Time [s]')
ax[0,0].legend(loc='best')
if split_generators:
    Nstd = 3.5
    ax[0,0].set_ylim(Pe[0]['G03a'].mean() + np.array([-1,1]) * Nstd * Pstd)
    ax[1,0].set_ylim(Pe[0]['G03b'].mean() + np.array([-1,1]) * Nstd * Pstd)
    ax[0,1].set_ylim(Qe[0]['G03a'].mean() + np.array([-1,1]) * Nstd * Qstd)
    ax[1,1].set_ylim(Qe[0]['G03b'].mean() + np.array([-1,1]) * Nstd * Qstd)
else:
    Nstd = 4
    ax[0,0].set_ylim(Pe[0]['G03'].mean() + np.array([-1,1]) * Nstd * Pstd)
    ax[0,1].set_ylim(Qe[0]['G03'].mean() + np.array([-1,1]) * Nstd * Qstd)
ax[0,0].set_xlim(xlim)
fig.tight_layout()
if split_generators:
    fig.savefig(f'IEEE39_PF_rnd_load_split_gen_PQ_gen_{S_G03a}_{S_G03b}{suffix}.pdf')
else:
    fig.savefig(f'IEEE39_PF_rnd_load_PQ_gen.pdf')

In [ ]:
if Iline is not None:
    line_IDs = ((10, 11), (10, 13), (14, 15), (16, 17), (1, 39), (3, 4))
    n_lines = len(line_IDs)
    n_cols = 2
    n_rows = n_lines // n_cols
    fig,ax = plt.subplots(n_rows, n_cols, sharex=True, figsize=(n_cols * 5, n_rows * 2))
    cmap = 'kr'
    for i,I in enumerate(Iline):
        for j,(a,b) in enumerate(line_IDs):
            m = j % n_rows
            n = j // n_rows
            key = f'{a}_{b}'
            ax[m,n].plot(time, I[key], lw=1, color=cmap[i], label=f'M = {M[i]:.2f}' + r' $GW\cdot s^2$')
            ax[m,n].set_ylabel(r'$I_{line~' + str(a) + '-' + str(b) + r'}$ [kA]')
            for side in 'right','top':
                ax[m,n].spines[side].set_visible(False)
            ax[m,n].grid(which='major', axis='y', lw=0.5, ls=':', color=[.6,.6,.6])
    xlim = [0, 60]
    for a in ax[-1,:]:
        a.set_xlabel('Time [s]')
    ax[0,0].legend(loc='best')
    ax[0,0].set_xlim(xlim)
    fig.tight_layout()
    if split_generators:
        fig.savefig(f'IEEE39_PF_rnd_load_split_gen_Ilines_{S_G03a}_{S_G03b}{suffix}.pdf')
    else:
        fig.savefig(f'IEEE39_PF_rnd_load_Ilines.pdf')

In [ ]:
bus_IDs = 3, 10, 14, 17, 39
n_bus = len(bus_IDs)
fig,ax = plt.subplots(n_bus, 2, sharex=True, figsize=(10, n_bus * 2))
cmap = 'kr'
for i,(vd,vq) in enumerate(zip(Vd, Vq)):
    for j,bus_ID in enumerate(bus_IDs):
        ax[j,0].plot(time, vd[bus_ID], lw=1, color=cmap[i], label=f'M = {M[i]:.2f}' + r' $GW\cdot s^2$')
        ax[j,1].plot(time, vq[bus_ID], lw=1, color=cmap[i])
        ax[j,0].set_ylabel(r'$Vd_{bus ' + str(bus_ID) + r'}$ [kV]')
        ax[j,1].set_ylabel(r'$Vq_{bus ' + str(bus_ID) + r'}$ [kV]')
        for side in 'right','top':
            ax[j,i].spines[side].set_visible(False)
        ax[j,i].grid(which='major', axis='y', lw=0.5, ls=':', color=[.6,.6,.6])
for a in ax[-1,:]:
    a.set_xlabel('Time [s]')
Nstd = 3.5
ax[0,0].legend(loc='best')
ax[0,0].set_xlim(xlim)
fig.tight_layout()
if split_generators:
    fig.savefig(f'IEEE39_PF_rnd_load_split_gen_Vd_Vq_{S_G03a}_{S_G03b}{suffix}.pdf')
else:
    fig.savefig(f'IEEE39_PF_rnd_load_Vd_Vq.pdf')

In [ ]:
# var_names = [var_name.format(bus_ID) for bus_ID in np.unique(rec_bus_IDs) for var_name in var_names_fmt]
# var_names_data = [
#     'Pe_line_3_4', 'Qe_line_3_4', 'Vd_bus3', 'Vq_bus3',
#     'Pe_line_14_15', 'Qe_line_14_15', 'Vd_bus14', 'Vq_bus14',
#     'Pe_line_16_17', 'Qe_line_16_17', 'Vd_bus17', 'Vq_bus17',
# ]
var_names_data = [
    'Pe_bus3', 'Qe_bus3', 'Vd_bus3', 'Vq_bus3',
    'Pe_bus14', 'Qe_bus14', 'Vd_bus14', 'Vq_bus14',
    'Pe_bus17', 'Qe_bus17', 'Vd_bus17', 'Vq_bus17',
]
var_names_network = [var_name.format(bus_ID) for bus_ID in np.unique(rec_bus_IDs) for var_name in var_names_fmt]
if split_generators:
    data_mean = None
else:
    data_mean = {var_name: x_train_mean[k] for k,var_name in enumerate(var_names_network)}
data_std = {var_name: x_train_std[k] for k,var_name in enumerate(var_names_network)}

window_dur = 60
window_step = 1

data = []
data_normalized = []
measure = []

replace_keys = lambda D, keys_in, keys_out: {key_out: D[key_in] for key_in, key_out in zip(keys_in, keys_out)}

for data_file in data_files:
    t, d, data_norm, data_sliding, _ = load_data_slide([data_file],
                                                        var_names_data,
                                                        data_mean,
                                                        data_std,
                                                        window_dur,
                                                        window_step,
                                                        add_omega_ref = False,
                                                        verbose = True)
    d = replace_keys(d, var_names_data, var_names_network)
    data_norm = replace_keys(data_norm, var_names_data, var_names_network)
    data_sliding = replace_keys(data_sliding, var_names_data, var_names_network)
    data.append(d)
    data_normalized.append(data_norm)
    dt = np.diff(t[:2])[0]
    time, pred, _ = predict(model, data_sliding, window_step)
    measure.append(pred)
measure_predicted = np.array(list(map(np.nanmean, measure)))
MAPE = np.mean(np.abs((M - measure_predicted) / M)) * 100

In [ ]:
fig,ax = plt.subplots(1, 1, figsize=(5,4))
ax.plot(M, M, 'k--', lw=1)
ax.plot(M, measure_predicted, 'ko', markerfacecolor='w', markersize=8, markeredgewidth=2)
for side in 'right','top':
    ax.spines[side].set_visible(False)
ax.grid(which='major', lw=0.5, ls=':', color=[.6,.6,.6])
ax.set_xlabel(r'Momentum [GW$\cdot s^2$]')
ax.set_ylabel(r'Predicted momentum [GW$\cdot s^2$]')
ax.text(M[0], M[1], f'MAPE = {MAPE:.2f}%')
ax.set_xlim([0.168, 0.232])
ax.set_ylim([0.148, 0.232])
fig.tight_layout()
if split_generators:
    fig.savefig(f'IEEE39_PF_rnd_load_split_gen_momentum_prediction_{S_G03a}_{S_G03b}{suffix}.pdf')
else:
    fig.savefig(f'IEEE39_PF_rnd_load_momentum_prediction.pdf')    

In [ ]:
data_mean = {var_name: x_train_mean[k] for k,var_name in enumerate(var_names_network)}
data_std = {var_name: x_train_std[k] for k,var_name in enumerate(var_names_network)}